# On the Identification of Pulsar Stars

A DSCI 100 Project

Lucas Kuhn, Sophia Zhang, Michael Cheung

## Introduction

Out in the universe, astronomers have found many neutron stars. A subset of these stars fall under a rare classification known as a "pulsar", meaning that they produce a periodic radio signal that is detectable from Earth. These stars are of great interest, and so being able to identify them is a useful skill that furthers exploration efforts in understanding neutron stars.

In this project, we set out to develop a model for classifying them distinctly from those that are "non-pulsar". Our question to answer: "Are there characteristics of a pulsar candidate's curve profiles that serve as good predictors for pulsar and non-pulsar classification?".

To investigate this, we use the HTRU2 dataset provided by the UCI Machine Learning Repository. This dataset contains summary statistics for observed neutron stars as they exhibit on both the Integrated Pulse (folded) Profile, as well as on the Dispersion Measure Signal-to-Noise Ratio (DM-SNR) curve. It contains a large number of pulsar candidates that have known classifications as either "pulsar" or "non-pulsar".

## Methods

// TODO: (copied from proposal) actually state what's happening and narrate through operations

// TODO: use references in this section + in-text citations?

We plan to train multiple models using various subsets of the summary-statistic columns as predictor variables, given how the pulses for a candidate that is truly pulsar are "remarkably constant over long periods of time" (Karastergiou et. al, 2011) and as such tend to reveal prominent insights on curve profiles. 

We can use scatter plots similar to above to qualitatively assess pairs of variables for any distinct separation or clustering of classified points. Plots like this provide an interface for human inspection, as used to train other models such as artificial neural networks (Eatough et al., 2010).

We will fit a K-Nearest Neighbours classifier model to our training data, using a GridSearchCV with 5-fold Cross-Validation to choose $k$. This allows us to efficiently test the validation accuracy at many values of $k$ and combat both overfitting and underfitting.

With our best model, we can assess its accuracy on the testing set using a confusion matrix, allowing us to more critically assess the accuracy based on whether false positives are more costly than false negatives.

## Results

// TODO: results

We found that ...

The columns that best help to train the classifier are ...

## Discussion

// TODO: discussion

We found that (columns that are important) were good classifiers...

These findings could imply... / The impact that this might have is ...

Future questions that this investigation could lead to include ...

## References

1. Eatough, R. P., Molkenthin, N., Kramer, M., Noutsos, A., Keith, M. J., Stappers, B. W., & Lyne, A. G. (2010). Selection of radio pulsar candidates using artificial neural networks. Monthly Notices of the Royal Astronomical Society, 407(4), 2443-2450.

2. Karastergiou, A., Roberts, S. J., Johnston, S., Lee, H. J., Weltevrede, P., & Kramer, M. (2011). A transient component in the pulse profile of PSR J0738− 4042. Monthly Notices of the Royal Astronomical Society, 415(1), 251-256.